In [2]:
import json

import numpy
import pandas

In [3]:
# Load
mpra_df = pandas.read_csv('enhancer_mpra_processed.tsv', sep='\t', index_col=0)
# Remove duplicated sequences
mpra_df = mpra_df.drop_duplicates(subset='sequence')
mpra_df

C:\Users\casti\AppData\Local\Temp\ipykernel_17508\495785815.py:2: DtypeWarning: Columns (6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  mpra_df = pandas.read_csv('enhancer_mpra_processed.tsv', sep='\t', index_col=0)


,sequence,category,source,target,tunable_setpoint,background_id,motif_id,motif_name,motif_consensus,motif_rc,...,baseMean_HMC3,log2FC_HMC3,lfcSE_HMC3,pval_HMC3,padj_HMC3,baseMean_Retina,log2FC_Retina,lfcSE_Retina,pval_Retina,padj_Retina
id,,,,,,,,,,,,,,,,,,,,,
control_neg_genomic_1.818679,TATGTTCGTATTAAAACTAAGAGGAGAGATAATCCAGCTATGTTAC...,control_negative,genome_dhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,813.981261,-0.249698,0.043508,2.144383e-21,1.022478e-20,813.981261,-1.591276,0.430170,6.862848e-07,0.000004
control_neg_genomic_18.21946,TATTTCAATTCTGGTATCCTCTGATCTTTACGTTTTCTTCTCATGA...,control_negative,genome_dhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62.393622,0.254584,0.235787,6.489914e-01,6.952142e-01,62.393622,2.185929,0.519908,2.239101e-05,0.000117
control_neg_genomic_22.966011,ATTGAGCTCTTGCTGCCATCTGGAATTCATTTTTGTGTGTGGTTTG...,control_negative,genome_dhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,282.562203,0.024049,0.088832,1.146879e-01,1.446148e-01,282.562203,0.000000,0.427888,3.991249e-01,0.533274
control_neg_genomic_1.2732422,TAGAGCTAAGAGCTGCTGCCTTGCAGGGCTGTTAAGCGTAGTAAAC...,control_negative,genome_dhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,203.239904,-0.355479,0.095512,2.891019e-08,7.136632e-08,203.239904,-0.364807,0.566560,1.301603e-01,0.236722
control_neg_genomic_2.613447,ACAATTCTGTCTCCCAATCTGCTTAGCATCTTTCCTTTTTCTCATA...,control_negative,genome_dhs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88.142144,0.270324,0.142378,4.214563e-01,4.747524e-01,88.142144,0.397089,0.571616,8.410717e-01,0.893304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
triple_motif_embedding_SKNSH;HepG2;K562_27,TTTAATCAAATTATATTTTCACTTTTTTTGGACTTTGGAAAAAAAC...,triple_motif_embedding,motif_embedding,"SKNSH, HepG2, K562",NaN,control_neg_shuffled_18.21946,"MA0681.2, MA0114.4, MA0140.2","PHOX2B, HNF4A, GATA1::TAL1","TTTAATCAAATTATAT, TTTGGACTTTGGA, CTGCTCCTCACAG...","False, True, True",...,533.330615,-0.021248,0.057926,1.468786e-03,2.417848e-03,533.330615,-0.315780,0.378522,6.106183e-02,0.133206
triple_motif_embedding_SKNSH;HepG2;K562_28,ATATAATTTGATTAAATTTCACTTTTTCCAAAGTCCAAAAAAAAAC...,triple_motif_embedding,motif_embedding,"SKNSH, HepG2, K562",NaN,control_neg_shuffled_18.21946,"MA0681.2, MA0114.4, MA0140.2","PHOX2B, HNF4A, GATA1::TAL1","ATATAATTTGATTAAA, TCCAAAGTCCAAA, CTTATCTGTGAGG...","True, False, False",...,781.926341,-0.391259,0.043110,5.290603e-38,3.646621e-37,781.926341,0.825595,0.182285,3.208651e-03,0.011444
triple_motif_embedding_SKNSH;HepG2;K562_29,ATATAATTTGATTAAATTTCACTTTTTCCAAAGTCCAAAAAAAAAC...,triple_motif_embedding,motif_embedding,"SKNSH, HepG2, K562",NaN,control_neg_shuffled_18.21946,"MA0681.2, MA0114.4, MA0140.2","PHOX2B, HNF4A, GATA1::TAL1","ATATAATTTGATTAAA, TCCAAAGTCCAAA, CTGCTCCTCACAG...","True, False, True",...,435.919807,-0.350391,0.061558,4.417880e-17,1.773466e-16,435.919807,-0.525940,0.416363,2.061589e-02,0.056015


In [4]:
# Split data into n_subsets randomly
n_subsets = 7
seq_partitions = numpy.array_split(mpra_df.sample(frac=1, random_state=0), n_subsets)

# Print # elements
for i, seq_partition in enumerate(seq_partitions):
    print(f"Partition {i}: {len(seq_partition)} sequences total.")

# Print category allocation for the first partition
display(seq_partitions[0].value_counts('category'))
display(seq_partitions[1].value_counts('category'))

Partition 0: 1279 sequences total.
Partition 1: 1279 sequences total.
Partition 2: 1278 sequences total.
Partition 3: 1278 sequences total.
Partition 4: 1278 sequences total.
Partition 5: 1278 sequences total.
Partition 6: 1278 sequences total.


category
single_fsp                263
single_den                239
single_genomic            185
single_fsp_tunable        153
triple_fsp                130
double_fsp                119
triple_motif_embedding     55
double_genomic             39
double_motif_embedding     29
single_motif_embedding     27
triple_genomic             27
control_yin2025             8
control_negative            5
dtype: int64

category
single_fsp                246
single_den                207
single_fsp_tunable        177
single_genomic            174
double_fsp                151
triple_fsp                137
double_genomic             54
double_motif_embedding     41
triple_genomic             31
single_motif_embedding     27
triple_motif_embedding     25
control_negative            5
control_yin2025             4
dtype: int64

In [5]:
# Construct split info dictionaries
splits_info = []

for partition_idx, seq_partition in enumerate(seq_partitions):
    split_info = {}

    # Current partition as test set
    test_ids = seq_partitions[partition_idx].index.to_list()

    # Combine all remaining partitions starting with the next partition
    # the first 1 / n_subsets / 1.5 fraction of the sequences are used as validation set
    # the remaining sequences are used as training set
    # This achieves a 50% smaller validation set compared to the test set
    train_val_partition = pandas.concat(
        [
            seq_partitions[i]
            for i in range(partition_idx + 1, len(seq_partitions))
        ] +\
        [
            seq_partitions[i]
            for i in range(0, partition_idx)
        ]
    )
    val_ids = train_val_partition.index[:int(1/n_subsets/1.5 * len(mpra_df))].to_list()
    train_ids = train_val_partition.index[int(1/n_subsets/1.5 * len(mpra_df)):].to_list()
    
    split_info['train'] = train_ids
    split_info['val'] = val_ids
    split_info['test'] = test_ids
    
    splits_info.append(split_info)

In [6]:
# Basic sanity checks
for split_idx, split_info in enumerate(splits_info):
    print(f"Split {split_idx}")
    
    train_ids = split_info['train']
    val_ids = split_info['val']
    test_ids = split_info['test']

    # Extract data
    mpra_train_df = mpra_df.loc[train_ids]
    mpra_val_df = mpra_df.loc[val_ids]
    mpra_test_df = mpra_df.loc[test_ids]

    print(f"Train: {len(mpra_train_df)} sequences")
    print(f"Val: {len(mpra_val_df)} sequences")
    print(f"Test: {len(mpra_test_df)} sequences")

    # assert that val and test ids are not in the training list
    assert len(set(train_ids) & set(val_ids)) == 0
    assert len(set(train_ids) & set(test_ids)) == 0
    assert len(set(val_ids) & set(test_ids)) == 0

    print()

Split 0
Train: 6817 sequences
Val: 852 sequences
Test: 1279 sequences

Split 1
Train: 6817 sequences
Val: 852 sequences
Test: 1279 sequences

Split 2
Train: 6818 sequences
Val: 852 sequences
Test: 1278 sequences

Split 3
Train: 6818 sequences
Val: 852 sequences
Test: 1278 sequences

Split 4
Train: 6818 sequences
Val: 852 sequences
Test: 1278 sequences

Split 5
Train: 6818 sequences
Val: 852 sequences
Test: 1278 sequences

Split 6
Train: 6818 sequences
Val: 852 sequences
Test: 1278 sequences



In [ ]:
# Save split info
with open('mpra_data_splits.json', 'w') as f:
    json.dump(splits_info, f)